# Cartes Choroplèthes avec Folium

Une carte *choroplèthe* est une carte thématique où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique, tels la densité de population ou le revenu par habitant. (Wikipedia)

La réalisation d'une carte choroplèthe nécessite de disposer :
- d'un fichier contenant les délimitations des régions que l'on souhaite colorer au format `GeoJSON`.
- d'un fichier de données contenant les informations que l'on souhaite représenter sur la carte. Ces données peuvent être contenues dans le fichier `GeoJSON` ou être à part dans un fichier `csv` par exemple, voire saisies manuellement dans le programme python si leur nombre est raisonnable. 

In [1]:
# import des bibliothèques nécessaires
import folium # pour les cartes
import json   # pour les limites des régions 
import pandas # pour les données exploitées par folium

---
## Exemple 1 : une carte des départements normands
#### Dans cet exemple on représente toutes les zones délimitées de la même façon, sans prendre en compte de données chiffrée.

On peut trouver les délimitations des départements français sur une base de données ouvertes : `https://public.opendatasoft.com/explore/dataset/contours-simplifies-des-departements-francais-2015`

Pour sélectionner une région administrative particulière (par exemple la Normandie), utiliser la colonne filtre et saisir la région concernée dans la barre de recherche.

Exporter les données au format GeoJSON, et nommer le fichier de façon explicite. 

On peut visualiser sur une carte les départements délimités par le fichier GeoJSON créé. 

Les couleurs de contours et de remplissage des régions sont soit des codes hexadécimaux, soit des noms de couleurs du web 
(consulter par exemple https://fr.wikipedia.org/wiki/Couleur_du_Web.)

In [2]:
#création d'une carte
carte = folium.Map(location = [49.2, 0], zoom_start = 8)

#chargement des données géographiques
with open('donnees/contours-simplifies-departements-normands-2015.geojson', encoding='utf-8') as fichier:
    geo_dept = json.load(fichier)

folium.Choropleth(geo_data = geo_dept,     # données géographiques
                  line_color='Brown',
                  line_weight = 2,         # epaisseur des tracés
                  fill_color = 'Chocolate',
                  fill_opacity = 0.5,      # variable, de 0 (sans remplissage) à 1 (opaque)
                  highlight = True         # optionnel, permet de mettre en évidence chaque région au survol
                  ).add_to(carte)

#carte.save('CarteDepartementsNormandie.html')
carte

#### Peut-on afficher sur la carte des informations complémentaires ?

Le fichier `GeoJSON` est un *dictionnaire*, qui contient lui-même des dictionnaires. 

Le dictionnaire principal est associé au type `FeatureCollection` et contient une liste de dictionnaires, chacun d'eux étant associé au type `Feature` (caractéristique, en français). 

Chacun des objets de type `Feature` (qui correspondent ici aux départements) contient une entrée dont la clé est `geometry`, et dont la valeur permet de délimiter la zone concernée à l'aide de coordonnées de géolocalisation.

Une autre entrée a pour clé `properties` et sa valeur est (encore) un dictionnaire. Il contient des données propres à chaque département.

Par exemple, pour le département de l'Eure, les propriétés du fichier sont : 

    "properties": {
	 		"y_centroid": 6888805.0, 
	 		"nom_dept": "EURE", 
	 		"nom_reg": "NORMANDIE", 
	 		"code_reg": "28",
	 		 "geo_point_2d": [49.11369455931731, 0.996296267055192], 
	 		 "code_dept": "27", 
	 		 "nom_chf": "EVREUX", 
	 		 "code_chf": "229", 
	 		 "id_geofla": "DEPARTEM0000000000000045", 
	 		 "x_chf_lieu": 564892.0, 
	 		 "x_centroid": 553891.0, 
	 		 "y_chf_lieu": 6881977.0
             }
             
On peut afficher les entrées de toutes les propriétés du premier département à l'aide de l'instruction :

In [3]:
for cle in geo_dept["features"][0]["properties"].keys():
    print(cle, end = "   ")

y_centroid   nom_dept   nom_reg   code_reg   geo_point_2d   code_dept   nom_chf   code_chf   id_geofla   x_chf_lieu   x_centroid   y_chf_lieu   style   highlight   

*Remarque* : `style` et `highlight` ont été ajoutés par Python à la variable `geo_dept`.

On peut ainsi accéder aux valeurs associées à chaque clé et afficher sur la carte celles que l'on souhaite. 

In [4]:
#definition des marqueurs
for departement in geo_dept["features"]:
    proprietes = departement["properties"]
    coordonnees = proprietes["geo_point_2d"]
    folium.Marker(coordonnees, 
                  #facultatif, message au survol 
                  tooltip = proprietes["nom_dept"] + " (" + proprietes["code_dept"] + ")",
                  # facultatif, icone spécifique
                  icon = folium.Icon(icon='crosshairs', prefix='fa', color='darkred')
                 ).add_to(carte)
carte

---
## Exemple 2 : une carte de la population dans les départements normands
#### Dans cet exemple on utilise un fichier GeoJSON contenant les données géographiques ET  les données chiffrées à représenter. 
#### L'objectif est que les départements soient affichés sur la carte d'une couleur qui dépend de la valeur chiffrée considérée. 

On peut trouver la population par département sur une base de données ouvertes : 
   https://public.opendatasoft.com/explore/dataset/population-francaise-par-departement-2018/table/

La région n'étant pas renseignée le filtrage des départements normands est moins simple. 
L'utilisation de la chaine `14 or 27 or 50 or 61 or 76` permet de ne sélectionner que les départements normands. 

Le fichier est à disposition. Il est nommé `population-normande-par-departement-2018.geojson`. 

On affiche ensuite les clés présentes dans les propriétés de chaque département. 

In [5]:
with open('donnees/population-normande-par-departement-2018.geojson', encoding='utf-8') as fichier:
    geo_dep_pop = json.load(fichier)

for cle in geo_dep_pop["features"][0]["properties"].keys():
    print(cle, end = "   ")

departement   code_departement   geo_point_2d   population   

La fonction suivante ne servira qu'à faciliter la lecture des données sur la carte.


In [6]:
def affiche(nb):
    # affiche le nombre nb en groupant par 3 les chiffres de la partie entière
    a =("{:,}".format(nb))
    return a.replace(","," ")

#### Lier des données à une carte

Par rapport à la carte prédécente, on ajoute un dataFrame qui est un objet de la bibliothèque `pandas`. Un dataframe se comporte comme un dictionnaire dont les clés sont les noms des colonnes et les valeurs sont des séries.

- L'une des données prises en compte est celle que l'on souhaite représenter : la population.
- L'autre donnée servira d'identifiant de chaque département : on choisit le code du département.

La clé (ou colonne)  "Departement" est associée à la liste des codes de tous les départements, et la clé "Population" est associée  à la liste des effectifs des populations de tous les départements du fichier.

Dans la construction de la carte choroplèthe, on rajoute les paramètres : 


    key_on = "feature.properties.code_departement",  
   pour spécifier que c'est le code du département qui sert à relier les données et la carte
    
    data = df,                                       
   pour indiquer le dataFrame à considérer
    
    columns = ["Departement", "Population"]          
   pour indiquer les colonnes du dataFrame à considérer


In [7]:
carte2 = folium.Map(location = [49.18, 0], zoom_start = 8)

#les marqueurs
for dept in geo_dep_pop["features"]:
    proprietes = dept["properties"]
    coordonnees = (proprietes["geo_point_2d"])
    folium.Marker(coordonnees, 
                  tooltip = proprietes["departement"] + " " + affiche((proprietes["population"])) + " hab."
                 ).add_to(carte2)
#le dataFrame    
df = pandas.DataFrame({
    "Departement" : [dept["properties"]["code_departement"] for dept in geo_dep_pop["features"]], 
    "Population" : [dept["properties"]["population"] for dept in geo_dep_pop["features"]]
})
 
#la carte choroplèthe
folium.Choropleth(geo_data = geo_dep_pop,
                  key_on = "feature.properties.code_departement",
                  data = df, 
                  columns = ["Departement", "Population"],
                  bins = 5, # nombre d'intervalles créés entre les valeurs min et max
                  line_color ='black',
                  line_weight = 1,         # epaisseur des tracés
                  fill_color = "OrRd",
                  fill_opacity = 0.5,      # variable de 0 (sans remplissage) à 1 (opaque)
                  highlight = True,
                  legend_name = "Population 2018"
                  ).add_to(carte2)  # echelle de couleurs à selectionner sur http://colorbrewer2.org

carte2

#### Mais d'où viennent les couleurs ?

Folium utilise les palettes de couleurs de Color Brewer suivantes: ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.

[Voir en ligne](http://colorbrewer2.org) pour plus d'informations. 

#### Peut-on choisir des intervalles non réguliers ?

Oui, il suffit d'affecter à `bins` une liste de valeurs, plutôt qu'un entier, par exemple :

`bins =[0, 400000, 600000, 975000, 1300000]`
                  
                 

#### Commentaires
En toute honnêteté, la carte précédente ne respecte pas la définition d'une carte choroplèthe. La donnée représentée n'est ni une valeur/habitant, ni une valeur par unité de surface. Une représentation avec des *figurés proportionnels* serait plus correcte pour ces données. 

L'exemple suivant concerne ainsi la carte de la densité de population. 


--- 
## Exemple 3. : une carte de la densité de population dans les départements normands
#### Dans cet exemple on utilise un fichier GeoJSON contenant les données géographiques et un fichier csv contenant  les données chiffrées à représenter. 
#### L'objectif est que les départements soient affichés sur la carte d'une couleur qui dépend de la valeur chiffrée considérée. 

Le fichier `csv` concerné est à disposition. Il est nommé `donnees_normandie_2018.csv`. 

Il contient la population, la surface et la densité de population par km$^2$, ainsi que le code et le nom du département. 

In [8]:
donnees = pandas.read_csv('donnees/donnees_normandie_2018.csv', sep = ';')

for colonne in donnees:
    print(colonne)

code
nom
population_2018
superficie_km2
densite_2018


In [14]:
#création d'une carte
carte3 = folium.Map(location = [49.2, 0], zoom_start = 8)

#les données géographiques : celles de l'exemple 1
with open('donnees/contours-simplifies-departements-normands-2015.geojson', encoding='utf-8') as fichier:
    geo_dept = json.load(fichier)
     
#le dataFrame
#donnees = pandas.read_csv('donnees/donnees_normandie_2018.csv', sep=';')
df = donnees[["code", "densite_2018"]]
df = df.astype({"code": str, 'densite_2018': float})


#les marqueurs
for departement in geo_dept["features"]:
    proprietes = departement["properties"]
    coordonnees = proprietes["geo_point_2d"]
    folium.Marker(coordonnees, 
                  tooltip = proprietes["nom_dept"], 
                  icon = folium.Icon(icon='crosshairs', prefix ='fa', color ='darkred')
                 ).add_to(carte3)

folium.Choropleth(geo_data = geo_dept,     # données géographiques
                  key_on = "feature.properties.code_dept",
                  data = df, 
                  columns = ["code", "densite_2018"],
                  bins = 4,                # nombre d'intervalles créés entre les valeurs min et max
                  fill_color = "OrRd",
                  fill_opacity = 0.5,      # variable de 0 (sans remplissage) à 1 (opaque)
                  highlight = True,
                  legend_name = "Densité de population 2018, en hab/km2"
                  ).add_to(carte3)

carte3

Source : https://python-visualization.github.io/folium/modules.html?highlight=choropleth#folium.features.Choropleth.render